In [ ]:
%load_ext jupyter_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tqdm
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import geopandas as gpd
from datetime import datetime
import rioxarray as rxr
from azure.storage.blob import ContainerClient
import xarray as xr
import matplotlib.pyplot as plt

load_dotenv()
ADMS = ["Sofala", "Inhambane", "Nampula", "Zambezia"]
AA_DATA_DIR = Path(os.getenv("AA_DATA_DIR"))
AA_DATA_DIR_NEW = Path(os.getenv("AA_DATA_DIR_NEW"))

DEV_BLOB_SAS = os.getenv("DSCI_AZ_SAS_DEV")
DEV_BLOB_NAME = "imb0chd0dev"
DEV_BLOB_URL = f"https://{DEV_BLOB_NAME}.blob.core.windows.net/"
DEV_BLOB_PROJ_URL = DEV_BLOB_URL + "projects" + "?" + DEV_BLOB_SAS
GLOBAL_CONTAINER_NAME = "global"
DEV_BLOB_GLB_URL = DEV_BLOB_URL + GLOBAL_CONTAINER_NAME + "?" + DEV_BLOB_SAS

dev_glb_container_client = ContainerClient.from_container_url(DEV_BLOB_GLB_URL)

## 1. Loading in raw data

In [ ]:
emdat_path = (
    AA_DATA_DIR_NEW
    / "private"
    / "processed"
    / "glb"
    / "emdat"
    / "emdat-tropicalcyclone-2000-2022-processed-sids.csv"
)

ibtracs_path = (
    Path(AA_DATA_DIR)
    / "public"
    / "raw"
    / "glb"
    / "ibtracs"
    / "IBTrACS.SI.list.v04r01.points/IBTrACS.SI.list.v04r01.points.shp"
)

adm1_path = (
    AA_DATA_DIR
    / "public"
    / "raw"
    / "moz"
    / "cod_ab"
    / "moz_admbnda_adm1_ine_20190607.shp"
)

df_emdat = pd.read_csv(emdat_path)
gdf_adm1 = gpd.read_file(adm1_path)
gdf_sel = gdf_adm1[gdf_adm1.ADM1_PT.isin(ADMS)]
gdf_ibtracs = gpd.read_file(ibtracs_path)

Clean up EM-DAT data to get list of impactful cyclones that hit landfall in our AOI in Mozambique. Combine with IBTrACS data.

In [ ]:
df_emdat_sel = df_emdat[df_emdat.iso3 == "MOZ"]
df_emdat_sel

In [ ]:
gdf_ibtracs["time"] = pd.to_datetime(gdf_ibtracs["ISO_TIME"])

for sid, group in gdf_ibtracs.groupby("SID"):
    print(sid)
    df_interp = group.set_index("time").resample("30min").interpolate().reset_index()

In [ ]:
gdf_ibtracs